In [9]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI
from langchain.schema import Document
import json

In [10]:
import os 

from app import apikey

os.environ["OPENAI_API_KEY"] = apikey


In [11]:
class JSONLoader:
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        with open(self.file_path, 'r') as f_in:
            docs_raw = json.load(f_in)
        
        documents = []
        for item in docs_raw:
            for doc_data in item.get("documents", []):
                # Assuming 'text' and 'course' are fields in your JSON
                text = doc_data.get('text', '')  
                course = item.get('course', '')  

                doc = Document(
                    page_content=text,
                    metadata={"course": course, "source": self.file_path}  # Adjust metadata fields as needed
                )
                documents.append(doc)
        
        return documents


In [12]:
json_loader = JSONLoader("documents.json")

In [13]:
documents = json_loader.load()

In [14]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [15]:
db = DocArrayInMemorySearch.from_documents(
    documents, 
    embeddings
)

In [16]:
retriever = db.as_retriever()

In [17]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

/home/codespace/.python/current/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [24]:
query = "when is the machine learning course starting"

In [25]:
response = qa_chain.run(query)

In [26]:
display(Markdown(response))

The Machine Learning course is scheduled to start in September 2024.